In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
np.random.seed(1)
tf.set_random_seed(1)
from tools import problems,networks,train
import math
import sys
import numpy.linalg as la

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # BE QUIET!!!! (info and warnings are not printed)

2022-07-14 12:30:00.141654: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Instructions for updating:
non-resource variables are not supported in the long term


### M, N, L, pnz, SNR, kappa = 250, 500, 1000, 0.1, 40, 0

# Bernoulli Gaussian Code

In [3]:
tf.compat.v1.disable_eager_execution()

In [4]:
#create an np matrix A and also a tf tensor (make it constant, since it need not be trained)
A = np.random.normal(size=(M, N), scale=1.0 / math.sqrt(M)).astype(np.float32)
A_ = tf.constant(A,namea='A')  #Made a tf constant since it need not be trained

Tf variables

In [5]:
#generate sparse x values drawn from gaussian iid, and non zero locations selected uniformly(with sparsity 0.1).
bernoulli_ = tf.cast(tf.random.uniform((N, L))<pnz, tf.float32) 
xgen_ = bernoulli_ * tf.random.normal((N,L)) 

noise_var = pnz* N/M * math.pow(10., -SNR/10.)
ygen_ = tf.matmul(A_,xgen_) + tf.random.normal( (M,L),stddev=math.sqrt( noise_var ) )

In [6]:
print(A_)

Tensor("A:0", shape=(250, 500), dtype=float32)


Np variables

In [7]:
xval = ((np.random.uniform( 0,1,(N,L))<pnz) * np.random.normal(0,1,(N,L))).astype(np.float32)
yval = np.matmul(A,xval) + np.random.normal(0,math.sqrt( noise_var ),(M,L))

xinit = ((np.random.uniform( 0,1,(N,L))<pnz) * np.random.normal(0,1,(N,L))).astype(np.float32)
yinit = np.matmul(A,xinit) + np.random.normal(0,math.sqrt( noise_var ),(M,L))

We have created 1000 instances of x and the correspoinding y's using the measurement matrix. Now that we have 1000 (x,y), we use this to train the unfolded neural network.

# Build LISTA 

In [8]:
T, initial_lambda = 6, 0.1

Shrinkage function

In [9]:
def simple_soft_threshold(r_, lam_):
    "implement a soft threshold function y=sign(r)*max(0,abs(r)-lam)"
    lam_ = tf.maximum(lam_, 0)
    return tf.sign(r_) * tf.maximum(tf.abs(r_) - lam_, 0)

In [10]:
# creating placeholders, which will hold dataset later.
x_ = tf.compat.v1.placeholder( tf.float32,(N,None),name='x' )
y_ = tf.compat.v1.placeholder( tf.float32,(M,None),name='y' )

In [11]:
eta = simple_soft_threshold #creating an object
B = A.T / (1.01 * la.norm(A,2)**2) #(This is according to original code, not sure why)
#B = A.T #numpy array

# All of these below are tf, which means they will be stored as graphs and will run later. 
# B and S are the trainable parameters
B_ = tf.Variable(B,dtype=tf.float32,name='B_0') #creating tf.variable to make it a trainable parameter. 
S_ = tf.Variable(np.identity(N) - np.matmul(B, A),dtype=tf.float32,name='S_0' )

By_ = tf.matmul(B_,y_)  # This will be the input for the shrinkage function in the first iteration

In [12]:
print(B_)
print(By_)

<tf.Variable 'B_0:0' shape=(500, 250) dtype=float32_ref>
Tensor("MatMul_1:0", shape=(500, ?), dtype=float32)


In [13]:
# not sure why it is written as a list. Might get clear later.
layers = []
layers.append( ('Linear',By_,None) ) 

In [14]:
initial_lambda = np.array(initial_lambda).astype(np.float32)
lam0_ = tf.Variable(initial_lambda,name='lam_0')  #to make it learnable

xhat_ = eta( By_, lam0_) #first itertion and xhat_0 is all 0s
layers.append( ('LISTA T=1',xhat_, (lam0_,) ) )

for t in range(1,T):
    lam_ = tf.Variable( initial_lambda,name='lam_{0}'.format(t) ) # using the same lambda
    xhat_ = eta( tf.matmul(S_,xhat_) + By_, lam_ )  # will be stored as graphs which will run during a session later.   
    layers.append( ('LISTA T='+str(t+1),xhat_,(lam_,)) ) # creating layers (xhat_ is an operation)         
    

In [15]:
print(*layers , sep = "\n")
print(lam0_)

('Linear', <tf.Tensor 'MatMul_1:0' shape=(500, ?) dtype=float32>, None)
('LISTA T=1', <tf.Tensor 'mul_1:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'lam_0:0' shape=() dtype=float32_ref>,))
('LISTA T=2', <tf.Tensor 'mul_2:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'lam_1:0' shape=() dtype=float32_ref>,))
('LISTA T=3', <tf.Tensor 'mul_3:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'lam_2:0' shape=() dtype=float32_ref>,))
('LISTA T=4', <tf.Tensor 'mul_4:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'lam_3:0' shape=() dtype=float32_ref>,))
('LISTA T=5', <tf.Tensor 'mul_5:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'lam_4:0' shape=() dtype=float32_ref>,))
('LISTA T=6', <tf.Tensor 'mul_6:0' shape=(500, ?) dtype=float32>, (<tf.Variable 'lam_5:0' shape=() dtype=float32_ref>,))
<tf.Variable 'lam_0:0' shape=() dtype=float32_ref>


In [16]:
for name,xhat_,var_list in layers:
    print(xhat_)

Tensor("MatMul_1:0", shape=(500, ?), dtype=float32)
Tensor("mul_1:0", shape=(500, ?), dtype=float32)
Tensor("mul_2:0", shape=(500, ?), dtype=float32)
Tensor("mul_3:0", shape=(500, ?), dtype=float32)
Tensor("mul_4:0", shape=(500, ?), dtype=float32)
Tensor("mul_5:0", shape=(500, ?), dtype=float32)
Tensor("mul_6:0", shape=(500, ?), dtype=float32)


# Setup training LISTA 

In [17]:
trinit,refinements,final_refine = 1e-3, (.5,.1,.01), None
losses_=[]
nmse_=[]
trainers_=[]

"""assert is used when, If the statement is true then it does nothing and continues 
the execution, but if the statement is False then it stops the execution of the program 
and throws an error.
"""
assert np.array(refinements).min()>0,'all refinements must be in (0,1]'
assert np.array(refinements).max()<=1,'all refinements must be in (0,1]'

tr_ = tf.Variable(trinit,name='tr',trainable=False) #Learning rate
training_stages=[]

In [18]:
tf.disable_v2_behavior() #compatibility

In [19]:
# # creating placeholders, which will hold dataset later.
# x_ = tf.compat.v1.placeholder( tf.float32,(N,None),name='x' )
# y_ = tf.compat.v1.placeholder( tf.float32,(M,None),name='y' )

In [20]:
print(x_)
print(y_)

Tensor("x:0", shape=(500, ?), dtype=float32)
Tensor("y:0", shape=(250, ?), dtype=float32)


In [21]:
nmse_denom_ = tf.nn.l2_loss(x_)  #computes half of l2 loss without the sqrt

In [22]:
for name,xhat_,var_list in layers:
    loss_  = tf.nn.l2_loss( xhat_ - x_) #original values of x will be stored in the placeholder
    nmse_  = tf.nn.l2_loss( (xhat_ - x_) ) / nmse_denom_ # computing the nmse.
    if var_list is not None:
            train_ = tf.train.AdamOptimizer(tr_).minimize(loss_, var_list=var_list) 
            training_stages.append( (name,xhat_,loss_,nmse_,train_,var_list) )
    for fm in refinements:
            train2_ = tf.train.AdamOptimizer(tr_*fm).minimize(loss_)
            training_stages.append( (name+' trainrate=' + str(fm) ,xhat_,loss_,nmse_,train2_,()) )
            
if final_refine:
    train2_ = tf.train.AdamOptimizer(tr_*final_refine).minimize(loss_)
    training_stages.append( (name+' final refine ' + str(final_refine) ,xhat_,loss_,nmse_,train2_,()) )

In [23]:
print(*training_stages , sep = "\n")

('Linear trainrate=0.5', <tf.Tensor 'MatMul_1:0' shape=(500, ?) dtype=float32>, <tf.Tensor 'L2Loss_1:0' shape=() dtype=float32>, <tf.Tensor 'truediv:0' shape=() dtype=float32>, <tf.Operation 'Adam' type=NoOp>, ())
('Linear trainrate=0.1', <tf.Tensor 'MatMul_1:0' shape=(500, ?) dtype=float32>, <tf.Tensor 'L2Loss_1:0' shape=() dtype=float32>, <tf.Tensor 'truediv:0' shape=() dtype=float32>, <tf.Operation 'Adam_1' type=NoOp>, ())
('Linear trainrate=0.01', <tf.Tensor 'MatMul_1:0' shape=(500, ?) dtype=float32>, <tf.Tensor 'L2Loss_1:0' shape=() dtype=float32>, <tf.Tensor 'truediv:0' shape=() dtype=float32>, <tf.Operation 'Adam_2' type=NoOp>, ())
('LISTA T=1', <tf.Tensor 'mul_1:0' shape=(500, ?) dtype=float32>, <tf.Tensor 'L2Loss_3:0' shape=() dtype=float32>, <tf.Tensor 'truediv_1:0' shape=() dtype=float32>, <tf.Operation 'Adam_3' type=NoOp>, (<tf.Variable 'lam_0:0' shape=() dtype=float32_ref>,))
('LISTA T=1 trainrate=0.5', <tf.Tensor 'mul_1:0' shape=(500, ?) dtype=float32>, <tf.Tensor 'L2Loss

# Do Training LISTA 

In [24]:
def save_trainable_vars(sess,filename,**kwargs):
    """save a .npz archive in `filename`  with
    the current value of each variable in tf.trainable_variables()
    plus any keyword numpy arrays.
    """
    save={}
    for v in tf.trainable_variables():
        save[str(v.name)] = sess.run(v)
    save.update(kwargs)
    np.savez(filename,**save)

In [25]:
def load_trainable_vars(sess,filename):
    """load a .npz archive and assign the value of each loaded
    ndarray to the trainable variable whose name matches the
    archive key.  Any elements in the archive that do not have
    a corresponding trainable variable will be returned in a dict.
    """
    other={}
    try:
        tv=dict([ (str(v.name),v) for v in tf.trainable_variables() ])
        for k,d in np.load(filename).items():
            if k in tv:
                print('restoring ' + k)
                sess.run(tf.assign( tv[k], d) )
            else:
                other[k] = d
    except IOError:
        pass
    return other

In [26]:
ivl, maxit, better_wait =10,1000000,5000
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('norms xval:{xval:.7f} yval:{yval:.7f}'.format(xval=la.norm(xval), yval=la.norm(yval) ) )

state = load_trainable_vars(sess,'LISTA_bg_giid.npz') # must load AFTER the initializer

    # must use this same Session to perform all training
    # if we start a new Session, things would replay and we'd be training with our validation set (no no)

done=state.get('done',[])
log=str(state.get('log',''))

for name,xhat_,loss_,nmse_,train_,var_list in training_stages:
    if name in done:
        print('Already did ' + name + '. Skipping.')
        continue
    if len(var_list):
        describe_var_list = 'extending ' + ','.join([v.name for v in var_list])
    else:
        describe_var_list = 'fine tuning all ' + ','.join([v.name for v in tf.trainable_variables() ])

    print(name + ' ' + describe_var_list)
    nmse_history=[]
    for i in range(maxit+1):
        if i%ivl == 0:
            nmse = sess.run(nmse_,feed_dict={y_:yval,x_:xval})
            if np.isnan(nmse):
                raise RuntimeError('nmse is NaN')
            nmse_history = np.append(nmse_history,nmse)
            nmse_dB = 10*np.log10(nmse)
            nmsebest_dB = 10*np.log10(nmse_history.min())
            sys.stdout.write('\ri={i:<6d} nmse={nmse:.6f} dB (best={best:.6f})'.format(i=i,nmse=nmse_dB,best=nmsebest_dB))
            sys.stdout.flush()
            if i%(100*ivl) == 0:
                print('')
                age_of_best = len(nmse_history) - nmse_history.argmin()-1 # how long ago was the best nmse?
                if age_of_best*ivl > better_wait:
                    break # if it has not improved on the best answer for quite some time, then move along
        #y,x = prob(sess)
        y, x= sess.run( ( ygen_,xgen_ ) ) # generates different y,x for every iteration.
        sess.run(train_,feed_dict={y_:y,x_:x} )
    done = np.append(done,name)

    log =  log+'\n{name} nmse={nmse:.6f} dB in {i} iterations'.format(name=name,nmse=nmse_dB,i=i)

    state['done'] = done
    state['log'] = log
    save_trainable_vars(sess,'LISTA_bg_giid.npz',**state)

2022-06-27 18:54:09.391416: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-27 18:54:09.392550: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-27 18:54:09.450037: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-06-27 18:54:09.450701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: Quadro P2200 computeCapability: 6.1
coreClock: 1.493GHz coreCount: 10 deviceMemorySize: 4.94GiB deviceMemoryBandwidth: 186.45GiB/s
2022-06-27 18:54:09.450727: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-0

norms xval:224.5516663 yval:224.0477230
Linear trainrate=0.5 fine tuning all B_0:0,S_0:0,lam_0:0,lam_1:0,lam_2:0,lam_3:0,lam_4:0,lam_5:0
i=0      nmse=-1.295706 dB (best=-1.295706)

2022-06-27 18:54:10.605730: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10



i=1000   nmse=-2.976344 dB (best=-2.978776)
i=2000   nmse=-2.978988 dB (best=-2.980044)
i=3000   nmse=-2.976944 dB (best=-2.980044)
i=4000   nmse=-2.976935 dB (best=-2.980044)
i=5000   nmse=-2.976858 dB (best=-2.980564)
i=6000   nmse=-2.976022 dB (best=-2.980564)
i=7000   nmse=-2.979383 dB (best=-2.980831)
i=8000   nmse=-2.977289 dB (best=-2.980831)
i=9000   nmse=-2.978209 dB (best=-2.980831)
i=10000  nmse=-2.978140 dB (best=-2.980831)
i=11000  nmse=-2.977339 dB (best=-2.980831)
i=12000  nmse=-2.976546 dB (best=-2.980831)
Linear trainrate=0.1 fine tuning all B_0:0,S_0:0,lam_0:0,lam_1:0,lam_2:0,lam_3:0,lam_4:0,lam_5:0
i=0      nmse=-2.976546 dB (best=-2.976546)
i=1000   nmse=-2.990776 dB (best=-2.991474)
i=2000   nmse=-2.990935 dB (best=-2.991608)
i=3000   nmse=-2.991336 dB (best=-2.992029)
i=4000   nmse=-2.990816 dB (best=-2.992029)
i=5000   nmse=-2.991111 dB (best=-2.992136)
i=6000   nmse=-2.990504 dB (best=-2.992136)
i=7000   nmse=-2.990688 dB (best=-2.992136)
i=8000   nmse=-2.99197

i=10000  nmse=-20.015743 dB (best=-20.030982)
i=11000  nmse=-20.010319 dB (best=-20.030982)
i=12000  nmse=-20.015428 dB (best=-20.031744)
i=13000  nmse=-20.007284 dB (best=-20.037755)
i=14000  nmse=-20.032868 dB (best=-20.037755)
i=15000  nmse=-19.987128 dB (best=-20.037755)
i=16000  nmse=-20.024278 dB (best=-20.037755)
i=17000  nmse=-20.001993 dB (best=-20.037755)
i=18000  nmse=-20.015213 dB (best=-20.040844)
i=19000  nmse=-20.002513 dB (best=-20.040844)
i=20000  nmse=-20.007796 dB (best=-20.040844)
i=21000  nmse=-19.984659 dB (best=-20.040844)
i=22000  nmse=-20.007679 dB (best=-20.040844)
i=23000  nmse=-19.978485 dB (best=-20.040844)
LISTA T=5 trainrate=0.01 fine tuning all B_0:0,S_0:0,lam_0:0,lam_1:0,lam_2:0,lam_3:0,lam_4:0,lam_5:0
i=0      nmse=-19.978485 dB (best=-19.978484)
i=1000   nmse=-20.108776 dB (best=-20.108775)
i=2000   nmse=-20.131454 dB (best=-20.132696)
i=3000   nmse=-20.136368 dB (best=-20.136624)
i=4000   nmse=-20.130713 dB (best=-20.136624)
i=5000   nmse=-20.132856 

# Testing  

In [27]:
xtest = ((np.random.uniform( 0,1,(N,1))<pnz) * np.random.normal(0,1,(N,1))).astype(np.float32)
ytest = np.matmul(A,xtest) + np.random.normal(0,math.sqrt( noise_var ),(M,1))

In [28]:
count = 0
for name,xhat_,var_list in layers:
    count = count + 1
    if count != 7:
        continue
    else:
        out = sess.run(xhat_,feed_dict={y_:ytest,x_:xtest})

In [29]:
print('xtest= \n',xtest)
print('estimated out \n', out)        



xtest= 
 [[-0.        ]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.25618178]
 [ 1.114737  ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.07645008]
 [ 0.        ]
 [-0.05123669]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-1.0751579 ]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.98249495]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 